# Homework 2

# Set up

## Installing packages

In [1]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [2]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [3]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 15.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 10.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 10.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 59.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 17.7MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [4]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio
nest_asyncio.apply()

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = asyncio.run(client.get_tools())
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

In [5]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

## A simple agent using tools from the MCP server


In [6]:
import os
import json
import datetime
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from typing import Dict, List, Optional, Tuple
import re
import asyncio
import json
from typing import Dict, Any, List, Optional, Tuple
from difflib import SequenceMatcher

import re
import json
from typing import Dict, Any, List, Tuple
from difflib import SequenceMatcher

async def verify_cv(cv_text: str, cv_name: str, tools) -> Dict[str, Any]:
    """
    Complete CV verification pipeline. Parses CV, searches social media,
    cross-checks against LinkedIn/Facebook, and returns scored report.

    Args:
        cv_text: Raw extracted text from CV PDF
        cv_name: CV filename (e.g. "CV1.pdf")
        tools:   mcptools list from notebook
                 tools[0] = searchfacebookusers
                 tools[1] = getfacebookprofile
                 tools[2] = getfacebookmutualfriends
                 tools[3] = searchlinkedinpeople
                 tools[4] = getlinkedinprofile
                 tools[5] = getlinkedininteractions

    Returns:
        Dict with score, discrepancies, and evidence
    """

    print(f"\n{'='*60}")
    print(f"📄 Verifying: {cv_name}")
    print(f"{'='*60}")

    # Step 1: Parse CV

    # Step 2: Search & retrieve social profiles (fixed tool calls)
    profiles = await search_social_profiles_fixed(
        cv_parsed,
        tools,
        name=cv_parsed.get("name"),
        locations=cv_parsed.get("locations"),
    )

    linkedin_profile = profiles.get("linkedin")
    facebook_profile = profiles.get("facebook")

    print("✓ LinkedIn profile:", "✓ Found" if linkedin_profile else "✗ Not found")
    print("✓ Facebook profile:", "✓ Found" if facebook_profile else "✗ Not found")

    if profiles.get("errors"):
        print("⚠️ Search warnings:", profiles["errors"])

    # Step 3: Verification checks
    discrepancies: List[Tuple[str, str]] = []

    # Check 0: missing profiles
    if not linkedin_profile and not facebook_profile and cv_parsed.get("name"):
        discrepancies.append(("MISSING_PROFILES", "No social media profiles found for verification"))
        print("⚠️ No social media profiles found for verification")

    print("\nRunning 6 verification checks...")

    # 1) Name consistency
    disc, reason = check_name(cv_parsed.get("name"), linkedin_profile, facebook_profile)
    if disc:
        discrepancies.append(("NAME", reason))
        print(f"✗ Name: {reason}")
    else:
        print("✓ Name check passed")

    # 2) Location consistency
    disc, reason = check_location(cv_parsed.get("locations"), linkedin_profile, facebook_profile)
    if disc:
        discrepancies.append(("LOCATION", reason))
        print(f"✗ Location: {reason}")
    else:
        print("✓ Location check passed")

    # 3) Education consistency
    disc, reason = check_education(cv_parsed.get("education"), linkedin_profile)
    if disc:
        discrepancies.append(("EDUCATION", reason))
        print(f"✗ Education: {reason}")
    else:
        print("✓ Education check passed")

    # 4) Experience consistency
    disc, reason = check_experience(cv_parsed.get("companies"), linkedin_profile)
    if disc:
        discrepancies.append(("EXPERIENCE", reason))
        print(f"✗ Experience: {reason}")
    else:
        print("✓ Experience check passed")

    # 5) Timeline feasibility
    disc, reason = check_timeline(cv_parsed.get("years"), linkedin_profile)
    if disc:
        discrepancies.append(("TIMELINE", reason))
        print(f"✗ Timeline: {reason}")
    else:
        print("✓ Timeline check passed")

    # Step 4: Score
    max_checks = 6  # 5 checks + missing profiles
    score = max(0.0, min(1.0, 1.0 - len(discrepancies) / max_checks))

    print("\n📊 Result:")
    print(f"   Discrepancies found: {len(discrepancies)}/{max_checks}")
    print(f"   Verification score: {score:.3f}")

    return {
        "cv_name": cv_name,
        "score": score,
        "discrepancies": discrepancies,
        "cv_parsed": {
            "name": cv_parsed.get("name"),
            "locations": cv_parsed.get("locations"),
            "companies": cv_parsed.get("companies"),
            "skills": cv_parsed.get("skills"),
            "education": cv_parsed.get("education"),
            "years": cv_parsed.get("years"),
        },
        "profiles": profiles,
    }

# =====================================
# Helper functions (all self-contained)
# =====================================

# ---------------------------
# Social search using MCP tools
# ---------------------------

def similarity_score(a: str, b: str) -> float:
    return SequenceMatcher(None, (a or "").lower(), (b or "").lower()).ratio()

async def search_social_profiles_fixed(
    cv_info: Dict[str, Any],
    tools,
    name: str = None,
    locations: List[str] = None,
) -> Dict[str, Any]:
    """
    Search LinkedIn + Facebook via mcptools list.
    """
    profiles = {"linkedin": None, "facebook": None, "errors": []}

    search_facebook_users_tool = tools[0]
    get_facebook_profile_tool = tools[1]
    search_linkedin_people_tool = tools[3]
    get_linkedin_profile_tool = tools[4]

    # LinkedIn Search
    try:
        li_candidates = await search_linkedin_people_tool.ainvoke({
            "q": name or "",
            "location": locations[0] if locations else None,
            "industry": None,
            "limit": 5,
            "fuzzy": True
        })

        if li_candidates and isinstance(li_candidates, list) and len(li_candidates) > 0:
            best_li = max(
                li_candidates,
                key=lambda p: similarity_score(p.get("name", ""), name or "")
            )

            raw_id = best_li.get("id")

            try:
                # If LangChain didn't inject a trace ID, this will safely cast to int
                person_id = int(raw_id)
            except (ValueError, TypeError):
                # If it's a trace ID ("lc_..."), we just use it as-is.
                # The underlying MCP server might reject it if it strictly wants an int,
                # but if LangChain is doing ID mapping behind the scenes, it will resolve it.
                person_id = raw_id

            try:
                # Call the underlying async function directly to bypass LangChain's strict schema validation
                li_profile = await get_linkedin_profile_tool.coroutine(personid=person_id)
            except TypeError:
                 # Some versions of LangChain map the underlying function with a dictionary
                li_profile = await get_linkedin_profile_tool.coroutine({"personid": person_id})

            profiles["linkedin"] = li_profile
            print(f"   ✓ LinkedIn best match: {best_li.get('name', 'N/A')}")

    except Exception as e:
        msg = f"LinkedIn search failed: {e}"
        profiles["errors"].append(msg)
        print("   " + msg)

    # Facebook Search
    try:
        fb_candidates = await search_facebook_users_tool.ainvoke({
            "q": name or "",
            "limit": 5,
            "fuzzy": True
        })

        if fb_candidates and isinstance(fb_candidates, list) and len(fb_candidates) > 0:
            best_fb = max(
                fb_candidates,
                key=lambda p: similarity_score(p.get("displayname", ""), name or "")
            )

            raw_id = best_fb.get("id")

            try:
                user_id = int(raw_id)
            except (ValueError, TypeError):
                user_id = raw_id

            try:
                fb_profile = await get_facebook_profile_tool.coroutine(userid=user_id)
            except TypeError:
                fb_profile = await get_facebook_profile_tool.coroutine({"userid": user_id})

            profiles["facebook"] = fb_profile
            print(f"   ✓ Facebook best match: {best_fb.get('displayname', 'N/A')}")

    except Exception as e:
        msg = f"Facebook search failed: {e}"
        profiles["errors"].append(msg)
        print("   " + msg)

    return profiles


# ---------------------------
# Consistency checks
# ---------------------------

def check_name(cv_name: str, li_profile: Dict, fb_profile: Dict) -> Tuple[bool, str]:
    if not cv_name:
        return False, ""
    names = []
    if li_profile:
        names.append(li_profile.get("name", ""))
    if fb_profile:
        names.append(fb_profile.get("displayname", ""))
        names.append(fb_profile.get("originalname", ""))
    for soc in names:
        if soc and similarity_score(cv_name, soc) > 0.8:
            return False, ""
    return True, f"CV name '{cv_name}' not found in social profiles"

def check_location(cv_locs: List[str], li_profile: Dict, fb_profile: Dict) -> Tuple[bool, str]:
    if not cv_locs:
        return False, ""
    soc_locs = []
    if li_profile:
        soc_locs.append(f"{li_profile.get('city', '')} {li_profile.get('country', '')}".lower())
    if fb_profile:
        soc_locs.append((fb_profile.get("city") or "").lower())
        soc_locs.append((fb_profile.get("country") or "").lower())
    cv_set = set(cv_locs)
    for sl in soc_locs:
        if any(loc in sl for loc in cv_set):
            return False, ""
    return True, f"CV locations {cv_locs} not found in social profiles"

def check_education(cv_edu: List[str], li_profile: Dict) -> Tuple[bool, str]:
    if not cv_edu or not li_profile:
        return False, ""
    liedu = [
        (e.get("school", "") + " " + e.get("degree", "")).lower()
        for e in li_profile.get("education", [])
    ]
    for edu in cv_edu:
        if any(edu in le for le in liedu):
            return False, ""
    return True, f"CV education {cv_edu} not found in LinkedIn"

def check_experience(cv_companies: List[str], li_profile: Dict) -> Tuple[bool, str]:
    if not cv_companies or not li_profile:
        return False, ""
    licompanies = [e.get("company", "").lower() for e in li_profile.get("experience", [])]
    found = any(comp.lower() in " ".join(licompanies) for comp in cv_companies)
    return (not found, f"CV companies {cv_companies} not found in LinkedIn experience")

def check_timeline(cv_years: List[int], li_profile: Dict) -> Tuple[bool, str]:
    if not cv_years:
        return False, ""
    cv_min, cv_max = min(cv_years), max(cv_years)

    if cv_max > 2026 or cv_min < 1950 or cv_max < cv_min:
        return True, f"Impossible timeline: {cv_min}-{cv_max}"

    if li_profile:
        li_years = li_profile.get("yearsexperience", 0)
        if li_years and abs(li_years - (cv_max - cv_min)) > 10:
            return True, f"Timeline mismatch: CV {cv_max - cv_min}y vs LinkedIn {li_years}y"

    return False, ""

# ---------------------------
# Main verify function
# ---------------------------

async def verify_cv(cv_text: str, cv_name: str, tools, cv_index: int = 0) -> Dict[str, Any]:
    print(f"\n{'='*60}")
    print(f"📄 Verifying: {cv_name}")
    print(f"{'='*60}")

    # ==========================================
    # Step 1: Agentic CV Parsing
    # ==========================================
    # Use the LLM to parse the CV instead of regex
    parse_prompt = f"""
    You are a CV parser. Please read the following CV text and extract key information.
    Return ONLY a valid JSON object with exactly these keys:
    - "name": (string) the person's full name
    - "locations": (list of strings) cities/countries mentioned
    - "companies": (list of strings) companies they worked for
    - "education": (list of strings) degrees or university names
    - "skills": (list of strings) professional skills
    - "years": (list of integers) any 4-digit years mentioned (e.g., 2020, 2024)

    CV Text:
    {cv_text}
    """

    # We use Gemini directly here to act as our parser
    parse_response = await llm.ainvoke([HumanMessage(content=parse_prompt)])

    # Clean up the JSON response
    clean_json = parse_response.content.replace("```json", "").replace("```", "").strip()
    try:
        cv_parsed = json.loads(clean_json)
    except json.JSONDecodeError:
        print("⚠️ Warning: LLM failed to return valid JSON. Falling back to empty dict.")
        cv_parsed = {
            "name": "", "locations": [], "companies": [],
            "education": [], "skills": [], "years": []
        }

    print("✓ Name extracted:", cv_parsed.get("name", "N/A"))
    print("✓ Locations found:", cv_parsed.get("locations", "N/A"))
    print("✓ Companies found:", cv_parsed.get("companies", "N/A"))
    print(
        "✓ Year range:",
        f"{min(cv_parsed.get('years', []))} - {max(cv_parsed.get('years', []))}"
        if cv_parsed.get("years") else "N/A"
    )

    # ==========================================
    # Step 2: Search Social Profiles
    # ==========================================
    profiles = await search_social_profiles_fixed(
        cv_parsed,
        tools,
        name=cv_parsed.get("name"),
        locations=cv_parsed.get("locations"),
    )

    linkedin_profile = profiles.get("linkedin")
    facebook_profile = profiles.get("facebook")

    print("✓ LinkedIn profile:", "✓ Found" if linkedin_profile else "✗ Not found")
    print("✓ Facebook profile:", "✓ Found" if facebook_profile else "✗ Not found")

    if profiles.get("errors"):
        print("⚠️ Search warnings:", profiles["errors"])

    # ==========================================
    # Step 3: Verification Checks
    # ==========================================
    discrepancies: List[Tuple[str, str]] = []

    if not linkedin_profile and not facebook_profile and cv_parsed.get("name"):
        discrepancies.append(("MISSING_PROFILES", "No social media profiles found for verification"))
        print("⚠️ No social media profiles found for verification")

    print("\nRunning 6 verification checks...")

    disc, reason = check_name(cv_parsed.get("name"), linkedin_profile, facebook_profile)
    if disc:
        discrepancies.append(("NAME", reason))
        print(f"✗ Name: {reason}")
    else:
        print("✓ Name check passed")

    disc, reason = check_location(cv_parsed.get("locations"), linkedin_profile, facebook_profile)
    if disc:
        discrepancies.append(("LOCATION", reason))
        print(f"✗ Location: {reason}")
    else:
        print("✓ Location check passed")

    disc, reason = check_education(cv_parsed.get("education"), linkedin_profile)
    if disc:
        discrepancies.append(("EDUCATION", reason))
        print(f"✗ Education: {reason}")
    else:
        print("✓ Education check passed")

    disc, reason = check_experience(cv_parsed.get("companies"), linkedin_profile)
    if disc:
        discrepancies.append(("EXPERIENCE", reason))
        print(f"✗ Experience: {reason}")
    else:
        print("✓ Experience check passed")

    disc, reason = check_timeline(cv_parsed.get("years"), linkedin_profile)
    if disc:
        discrepancies.append(("TIMELINE", reason))
        print(f"✗ Timeline: {reason}")
    else:
        print("✓ Timeline check passed")

    # ==========================================
    # Step 4: Scoring
    # ==========================================
    max_checks = 6
    score = max(0.0, min(1.0, 1.0 - len(discrepancies) / max_checks))

    print("\n📊 Result:")
    print(f"   Discrepancies found: {len(discrepancies)}/{max_checks}")
    print(f"   Verification score: {score:.3f}")

    return {
        "cv_name": cv_name,
        "score": score,
        "discrepancies": discrepancies,
        "cv_parsed": cv_parsed,
        "profiles": profiles,
    }


# # ---------------------------
# # 1. Define local tools
# # ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

@tool
async def verify_cv_tool(cv_index: int) -> str:
    """
    Verify a single CV by its 0-based index in the `all_cvs` global list.
    Parses the CV text, searches social media for inconsistencies, and returns a detailed report.

    Args:
        cv_index: The 0-based index of the CV to verify. For example, use `0` for CV_1.pdf, `1` for CV_2.pdf, etc.
    Returns:
        A JSON string of the verification report including score and discrepancies.
    """
    if not (0 <= cv_index < len(all_cvs)):
        return json.dumps({"error": f"Invalid CV index: {cv_index}. Must be between 0 and {len(all_cvs) - 1}."})

    cv_data = all_cvs[cv_index]
    cv_file = cv_data["file"]
    cv_text = cv_data["text"]

    try:
        report = await verify_cv(cv_text, cv_file, tools) # Calls the working verify_cv function
        return json.dumps(report, indent=2)
    except Exception as e:
        return json.dumps({"error": f"Failed to verify CV {cv_file}: {str(e)}"}) # Provide error message

@tool
def get_cv_text(cv_index: int) -> str:
    """
    Retrieve the parsed text of a CV by its index (0 to 4).
    Use index 0 for CV1.pdf, index 1 for CV2.pdf, etc.
    Returns a JSON string containing the filename and raw text.
    """
    try:
        idx = int(cv_index)
        if 0 <= idx < len(all_cvs):
            return json.dumps(all_cvs[idx])
        return json.dumps({"error": f"Invalid index. Must be 0 to {len(all_cvs)-1}"})
    except Exception as e:
        return json.dumps({"error": str(e)})

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

# mcp_tools is assumed to be globally available from cell 6h0311KbN9A3
# tools = mcp_tools + [verify_cv_tool]
tools = mcp_tools + [say_hello, verify_cv_tool, get_cv_text] # Add the new verify_cv_tool

# ---------------------------
# 3. Initialize Gemini (tool-enabled)
# ---------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_VERTEX_API_KEY,
    temperature=0,
)

llm_with_tools = llm.bind_tools(tools)

# The single-step invocation example is removed from here as the main demonstration
# will be in cell I8ifMpsVh6IQ for the conversational agent.

In [7]:
# This block provides you some tests to get faminilar with our MCP server

# Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alice", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# # Test 8: Get LinkedIn interactions
# await tools[5].ainvoke({'person_id': 456})

In [8]:
SYSTEM_PROMPT =  """
You are an autonomous AI assistant for automated CV verification.

Goal:
1. Fetch a candidate's CV text using the `get_cv_text` tool.
2. Read and parse the CV text internally to extract: Name, Location, Companies, Education, and Timeline (years).
3. Search LinkedIn (`searchlinkedinpeople`) using the extracted Name and Location.
4. If a LinkedIn match is found, extract its `id` and get the full profile using `getlinkedinprofile`.
5. Search Facebook (`searchfacebookusers`) using the extracted Name.
6. If a Facebook match is found, extract its `id` and get the full profile using `getfacebookprofile`.
7. Cross-check the CV claims against the retrieved social profiles.
8. Output a strictly formatted JSON verification report.

Available Tools:
- get_cv_text(cv_index)
- searchfacebookusers(q, limit, fuzzy)
- getfacebookprofile(userid)
- searchlinkedinpeople(q, location, industry, limit, fuzzy)
- getlinkedinprofile(personid)
- getlinkedininteractions(personid)

Important Rules for Tool Calling:
- When calling `getlinkedinprofile`, pass the argument exactly as `personid`.
- When calling `getfacebookprofile`, pass the argument exactly as `userid`.
- If multiple profiles are returned from a search, choose the best match based on name and location.
- If a search returns no results, do not make up fake profiles. State that it was not found.

Output Format:
Your final output MUST be a single JSON block formatted exactly like this:
{
  "cv_name": "CV1.pdf",
  "overall_score": 0.0,
  "evidence": {
    "cv_summary": { ... },
    "linkedin": { ... },
    "facebook": { ... }
  },
  "checks": { ... },
  "discrepancies": [ ... ]
}

Scoring:
Start at 1.0. Subtract 0.25 for High severity discrepancies, 0.15 for Medium, and 0.05 for Low. Floor at 0.0.
"""



In [9]:
async def agent_verify(cv_index: int) -> str:
    """Run the conversational agent to verify a single CV."""
    print(f"\n{'='*60}")
    print(f"🤖 Starting Agent Verification for CV {cv_index+1}")
    print(f"{'='*60}\n")

    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=f"Fetch and verify CV at index {cv_index}. Output the final JSON report.")
    ]

    max_iterations = 10

    for i in range(max_iterations):
        response = await llm_with_tools.ainvoke(messages)
        messages.append(response)

        # If the LLM didn't call tools, it's done (returning the final JSON)
        if not response.tool_calls:
            print(f"✅ Final Agent Output received.\n")
            return response.content

        # Execute requested tools
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]
            print(f"🔧 Agent calling tool: {tool_name}")
            # print(f"   Args: {tool_args}")

            # Find the actual tool object
            tool_obj = next((t for t in tools if t.name == tool_name), None)

            if tool_obj:
                try:
                    # LangChain handles the dict injection correctly here
                    result = await tool_obj.ainvoke(tool_args)
                    messages.append(ToolMessage(content=str(result), tool_call_id=tool_call["id"]))
                except Exception as e:
                    print(f"❌ Error in {tool_name}: {e}")
                    messages.append(ToolMessage(content=f"Error: {str(e)}", tool_call_id=tool_call["id"]))
            else:
                messages.append(ToolMessage(content=f"Tool {tool_name} not found.", tool_call_id=tool_call["id"]))

    return "Agent reached max iterations without completing."

In [10]:
async def process_all_cvs(all_cvs, tools):
    scores, results = [], []
    for i, cv in enumerate(all_cvs):
        # We pass cv_index=i just in case you need it later, though the text is passed directly
        res = await verify_cv(cv["text"], cv["file"], tools, cv_index=i)
        results.append(res)
        scores.append(res["score"])
    return scores, results

# Example execution:
final_scores, results = await process_all_cvs(all_cvs, tools)
print("Final Scores:", final_scores)



📄 Verifying: CV_1.pdf
✓ Name extracted: John Smith
✓ Locations found: ['Singapore', 'Kowloon']
✓ Companies found: ['ByteDance']
✓ Year range: 2009 - 2020
   LinkedIn search failed: 2 validation errors for call[get_linkedin_profile]
person_id
  Missing required argument [type=missing_argument, input_value={'personid': 'lc_d1c32504...42f3-95a2-87ac8b5a4d6e'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing_argument
personid
  Unexpected keyword argument [type=unexpected_keyword_argument, input_value='lc_d1c32504-12c7-42f3-95a2-87ac8b5a4d6e', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/unexpected_keyword_argument
   Facebook search failed: 2 validation errors for call[get_facebook_profile]
user_id
  Missing required argument [type=missing_argument, input_value={'userid': 'lc_43f2f5db-c...40b5-9662-ca517b491e30'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/miss

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [11]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [12]:
scores = final_scores
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [0, 0, 0, 0, 0], 'correct': 2, 'total': 5, 'final_score': 0.4}
